In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
def read_training_images(train_files, train_dir):
    """
    A method that reads the training image files in a given directory and returns the list of all images as numpy arrays
    """

    images=[]
    n=0
    for img_name in train_files.file_name:
        image_path = os.path.join(train_dir, img_name)
        img = cv2.imread(image_path)
        # img = np.expand_dims(cv2.resize(img, dsize=(COLS, ROWS), interpolation=cv2.INTER_CUBIC), axis=2)
        # .flatten()
        # print(n)
        # print(type(img))
        # print(img.shape)
        # cv2.imshow('image', img)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        img = img.astype('float32')
        images.append(img)
        n+=1
    print("Number of training images read = {}".format(n))

    return images

In [3]:
# Initialize the directories
root_dir = os.path.abspath('./')
train_dir = os.path.join(root_dir, 'Training')

train_files = pd.read_csv(os.path.join(root_dir, 'train_files.csv'))

# read the training and testing images
training_images = read_training_images(train_files, train_dir)


X_train = np.stack(training_images)
y_train = train_files.annotation.values



Number of training images read = 896


In [4]:
# Recreate the exact same model, including its weights and the optimizer
saved_model = tf.keras.models.load_model('./best_model/best_model.h5')

# Show the model architecture
saved_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 15, 20, 512)       14714688  
_________________________________________________________________
flatten (Flatten)            (None, 153600)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               78643712  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               1

In [5]:
y_pred = saved_model.predict(X_train, batch_size=16)

In [6]:
y_pred

array([[2.9698772e-09, 1.3528326e-06, 1.9556006e-02, 9.8035663e-01,
        8.5840351e-05],
       [4.3936285e-01, 2.5055090e-01, 1.9970720e-01, 6.1963215e-02,
        4.8415832e-02],
       [1.0000000e+00, 1.2924688e-14, 1.7643899e-10, 1.1333112e-25,
        1.6806203e-19],
       ...,
       [1.1283535e-04, 9.9787807e-01, 8.9279941e-04, 1.1059310e-03,
        1.0330981e-05],
       [2.8423717e-09, 1.0000000e+00, 4.9807717e-08, 1.5523721e-09,
        1.3857580e-13],
       [1.3710091e-01, 4.8878533e-01, 1.7591909e-01, 1.4526516e-01,
        5.2929498e-02]], dtype=float32)

In [8]:
np.savetxt('y_pred.csv', y_pred, delimiter=",")

In [9]:
y_train_cat = tf.keras.utils.to_categorical(y_train, num_classes=5)
np.savetxt('y_train.csv', y_train_cat, fmt="%d", delimiter=",")